In [2]:
import tiktoken

print(tiktoken.__version__)

0.12.0


## Exercise 2.1 Byte-pair encoding

In [4]:
tokenizer = tiktoken.get_encoding("gpt2")

In [6]:
uk_word = "Awirw ier"

integers = tokenizer.encode(uk_word)

In [9]:
for i in integers:
    print(f"{i} --> {tokenizer.decode([i])}")



23155 --> Aw
343 --> ir
86 --> w
220 -->  
959 --> ier


## Exercise 2.2

To develop more intuition for how the data loader works, try to run it with different settings such as max_length=2 and stride=2, and max_length=8 and stride=2.


In [10]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()